<a href="https://colab.research.google.com/github/donigagahhh/BIGDATA/blob/master/1800018196.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**PERHITUNGAN JUMLAH 10 APLIKASI TERATAS YANG PALING BANYAK DI INSTALL BERDASARKAN GENRE ATAU KATEGORI**


NAMA : DONI GAGAH PRAMUDYA
NIM : 1800018196

INSTALL SPARK PADA GOOGLE COLAB

In [ ]:
! apt-get install openjdk-8-jdk-headless -qq > /dev/null
! wget -q https://downloads.apache.org/spark/spark-2.4.6/spark-2.4.6-bin-hadoop2.7.tgz
! tar xf spark-2.4.6-bin-hadoop2.7.tgz
! pip install -q findspark
! pip install pyspark_dist_explore